<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=d3dbb4c0dbcedfca684d259fa3f15f183a6774d6ff4876143eb237e92ebb35dc
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2


    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-19 15:23:40
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  66.76 K
Current:  1.40 C
-------------------
Today PnL: 1.07 L (0.77%)
Current PnL: -26.74 L (-17.27%)
CY Booked + Current PnL: -12.45 L (-8.04%)
-------------------
Total profit:  1.41 L
Total loss:  -28.15 L
-------------------
Total Booked + Current PnL: 14.53 L (11.37%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.18%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.32 C
Est FTT PnL: 91.37 L (65.12%)
Deployed:  1.28 C
Current:  1.40 C
CAGR/XIRR %: 5.17%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,0.46,-19.78,24.78,0.09,20032.0,-19937.0,80840.0,87.62,30.0,M-SC,2.32,253.0,-1.00,0.58,6.21,OX40N,NTT,DURABLES
82,VOLTAS,1530.00,-1.78,7.71,11.13,19.69,22986.0,14778.0,206520.0,-2.74,52.0,X-MC,2.51,78.0,0.64,1.48,15.03,XY25,NTT,AC
50,MASFIN,398.61,1.50,-2.88,25.67,22.05,24428.0,-2820.0,95160.0,-16.44,52.0,H-SC,6.41,164.0,-0.12,0.68,38.32,XR,ATH,FINANCE
39,INDIGOPNTS,1408.00,0.83,-15.57,18.47,0.02,27220.0,-27185.0,147374.0,124.89,44.0,M-SC,13.14,234.0,-1.00,1.06,28.19,OX40N,NTT,PAINTS
43,ITC,452.00,0.16,-1.71,12.70,10.78,30000.0,-4100.0,236218.0,-41.56,45.0,X-LC,2.15,5.0,-0.14,1.69,3.44,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,RAJESHEXPO,518.0,9.03,-60.20,151.46,0.07,82682.0,-82587.0,54590.0,1901.26,67.0,L-SC,2.62,268.0,-1.00,0.39,35.77,OX40N,NTT,JEWELLERY
72,TATAELXSI,9161.0,8.02,-21.95,69.16,32.03,71161.0,-28941.0,102894.0,-15.53,62.0,H-SC,9.52,148.0,-0.41,0.74,15.25,OX40N,NTT,IT
18,COFFEEDAY,80.0,5.29,-39.80,124.72,35.27,85249.0,-45197.0,68352.0,-53.72,50.0,L-SC,10.43,270.0,-0.53,0.49,67.29,XR,NTT,HOTELS
6,ASIANTILES,137.0,4.94,-4.43,90.09,81.67,80770.0,-4155.0,89655.0,8072.22,71.0,L-SC,13.01,271.0,-0.05,0.64,73.62,XR,NTT,CERAMICS
64,SHALBY,327.0,3.67,-23.87,64.32,25.10,88702.0,-43236.0,137907.0,914.35,42.0,M-SC,10.97,250.0,-0.49,0.99,10.52,XY24,NTT,HEALTHCARE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
71,TANLA,1943.92,-5.48,-39.72,264.47,119.69,431628.0,-107556.0,163205.0,-39.22,37.0,H-SC,7.57,156.0,-0.25,1.17,28.93,AR,ATH,IT
38,INDIAMART,4810.62,-2.11,-4.76,117.07,106.72,137509.0,-5877.0,117459.0,-52.92,36.0,H-SC,3.68,139.0,-0.04,0.84,17.50,AR,ATH,MISC
54,PGHH,17907.65,-1.91,-4.51,40.53,34.19,82636.0,-9632.0,203888.0,-32.91,44.0,X-MC,5.23,57.0,-0.12,1.46,1.44,X40,ATH,FMCG
82,VOLTAS,1530.00,-1.78,7.71,11.13,19.69,22986.0,14778.0,206520.0,-2.74,52.0,X-MC,2.51,78.0,0.64,1.48,15.03,XY25,NTT,AC
61,ROUTE,2227.21,-1.36,-45.35,209.33,69.05,152225.0,-60346.0,72720.0,-55.87,56.0,H-SC,21.95,140.0,-0.40,0.52,12.15,SR,ATH,IT


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,1.57,-0.32,108.58,107.91,159796.0,-475.0,147169.0,-18.93,61.0,M-SC,12.26,216.0,-0.0,1.05,9.73,OX40N,ATH,CABLES


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,1.57,-0.32,108.58,107.91,159796.0,-475.0,147169.0,-18.93,61.0,M-SC,12.26,216.0,-0.00,1.05,9.73,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,0.83,-15.57,18.47,0.02,27220.0,-27185.0,147374.0,124.89,44.0,M-SC,13.14,234.0,-1.00,1.06,28.19,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,0.46,-19.78,24.78,0.09,20032.0,-19937.0,80840.0,87.62,30.0,M-SC,2.32,253.0,-1.00,0.58,6.21,OX40N,NTT,DURABLES
72,TATAELXSI,9161.00,8.02,-21.95,69.16,32.03,71161.0,-28941.0,102894.0,-15.53,62.0,H-SC,9.52,148.0,-0.41,0.74,15.25,OX40N,NTT,IT
66,SIS,528.00,0.79,-23.65,58.82,21.26,49865.0,-26257.0,84775.0,2000.52,53.0,H-SC,4.83,166.0,-0.53,0.61,14.54,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.0,0.26,8.5,58.77,72.27,108047.0,14400.0,183848.0,-6.99,70.0,M-LC,3.06,99.0,0.13,1.32,15.2,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,0.26,8.50,58.77,72.27,108047.0,14400.0,183848.0,-6.99,70.0,M-LC,3.06,99.0,0.13,1.32,15.20,XR,NTT,IT
25,FINCABLES,1641.55,1.57,-0.32,108.58,107.91,159796.0,-475.0,147169.0,-18.93,61.0,M-SC,12.26,216.0,-0.00,1.05,9.73,OX40N,ATH,CABLES
38,INDIAMART,4810.62,-2.11,-4.76,117.07,106.72,137509.0,-5877.0,117459.0,-52.92,36.0,H-SC,3.68,139.0,-0.04,0.84,17.50,AR,ATH,MISC
6,ASIANTILES,137.00,4.94,-4.43,90.09,81.67,80770.0,-4155.0,89655.0,8072.22,71.0,L-SC,13.01,271.0,-0.05,0.64,73.62,XR,NTT,CERAMICS
85,ZYDUSLIFE,1286.17,0.63,-3.04,39.88,35.63,81035.0,-6376.0,203198.0,-16.74,41.0,H-MC,2.64,119.0,-0.08,1.46,13.12,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,PAGEIND,51605.07,0.14,-11.26,44.37,28.11,79300.0,-22685.0,178725.0,-35.73,25.0,X-MC,13.79,55.0,-0.29,1.28,0.14,X40,ATH,APPARELS
9,BAJAJHFL,181.50,0.31,-18.34,90.39,55.47,179576.0,-44618.0,198668.0,-27.60,28.0,X-MC,14.37,64.0,-0.25,1.42,1.79,X40N,ATH,FINANCE
3,ACC,3906.00,-0.19,-26.39,123.01,64.15,215452.0,-62801.0,175150.0,-56.75,30.0,X-SC,3.22,82.0,-0.29,1.25,0.00,XY24,BTT,CEMENT
20,DABUR,735.00,0.21,-3.50,48.91,43.70,117089.0,-8672.0,239396.0,-11.44,35.0,X-MC,2.03,72.0,-0.07,1.71,11.03,XY24,BTT,FMCG
33,HINDUNILVR,2922.00,0.78,-11.18,27.98,13.67,63884.0,-28743.0,228320.0,-17.78,35.0,X-LC,2.60,14.0,-0.45,1.63,7.79,XY25,NTT,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,CAMS,950.0,0.76,-0.68,25.33,24.48,63360.0,-1716.0,250140.0,-81.09,48.0,X-SC,0.75,86.0,-0.03,1.79,22.30,X40N,NTT,MISC
12,BERGEPAINT,680.0,0.31,-4.16,26.39,21.14,57501.0,-9449.0,217890.0,-10.87,40.0,X-MC,1.34,74.0,-0.16,1.56,22.81,XY24,NTT,PAINTS
20,DABUR,735.0,0.21,-3.50,48.91,43.70,117089.0,-8672.0,239396.0,-11.44,35.0,X-MC,2.03,72.0,-0.07,1.71,11.03,XY24,BTT,FMCG
43,ITC,452.0,0.16,-1.71,12.70,10.78,30000.0,-4100.0,236218.0,-41.56,45.0,X-LC,2.15,5.0,-0.14,1.69,3.44,X40,NTT,FMCG
78,UNITDSPR,1644.0,1.07,3.29,16.92,20.77,40921.0,7705.0,241849.0,-5.65,45.0,X-MC,2.27,62.0,0.19,1.73,10.44,X40N,NTT,BREWERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,3906.00,-0.19,-26.39,123.01,64.15,215452.0,-62801.0,175150.0,-56.75,30.0,X-SC,3.22,82.0,-0.29,1.25,0.00,XY24,BTT,CEMENT
53,PAGEIND,51605.07,0.14,-11.26,44.37,28.11,79300.0,-22685.0,178725.0,-35.73,25.0,X-MC,13.79,55.0,-0.29,1.28,0.14,X40,ATH,APPARELS
59,RELAXO,1176.00,-0.15,-47.92,192.46,52.31,145490.0,-69565.0,75595.0,-44.54,41.0,X-SC,7.29,91.0,-0.48,0.54,1.19,X40N,NTT,FOOTWEAR
19,COLPAL,3726.84,1.08,-19.79,76.42,41.51,161437.0,-52115.0,211250.0,-4.54,41.0,X-MC,7.61,61.0,-0.32,1.51,1.20,XY25,ATH,FMCG
11,BATAINDIA,2096.00,1.22,-38.62,120.27,35.20,94988.0,-49691.0,78979.0,3.54,36.0,X-SC,16.58,92.0,-0.52,0.57,1.30,X40,NTT,FOOTWEAR


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,1.74,-29.49,102.59,42.84,47022.0,-19171.0,45835.0,-53.72,45.0,X-SC,34.76,83.0,-0.41,0.33,3.70,XY24,NTT,MISC
59,RELAXO,1176.00,-0.15,-47.92,192.46,52.31,145490.0,-69565.0,75595.0,-44.54,41.0,X-SC,7.29,91.0,-0.48,0.54,1.19,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,1.22,-38.62,120.27,35.20,94988.0,-49691.0,78979.0,3.54,36.0,X-SC,16.58,92.0,-0.52,0.57,1.30,X40,NTT,FOOTWEAR
51,MEDANTA,1486.00,1.66,-2.17,26.82,24.08,33310.0,-2750.0,124200.0,-10.14,43.0,X-SC,5.53,89.0,-0.08,0.89,17.02,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,0.52,-16.99,72.65,43.32,98223.0,-27672.0,135200.0,-28.68,37.0,X-SC,5.07,90.0,-0.28,0.97,3.81,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,-0.02,-9.81,33.84,20.71,115435.0,-37117.0,341120.0,-22.33,68.0,X-LC,3.88,1.0,-0.32,2.44,13.98,X40,ATH,IT
41,INFY,1972.00,0.84,11.67,20.21,34.24,70615.0,36517.0,349405.0,-13.14,69.0,X-LC,4.89,2.0,0.52,2.50,20.92,X40,NTT,IT
75,TMPV,600.00,2.14,-27.61,69.88,22.98,114276.0,-62366.0,163532.0,-24.30,42.0,X-LC,6.81,3.0,-0.55,1.17,2.85,XY24,NTT,AUTO
81,VBL,671.64,-0.58,-4.89,42.64,35.67,128091.0,-15440.0,300402.0,-16.24,50.0,X-LC,3.94,4.0,-0.12,2.15,8.29,X40N,ATH,FMCG
43,ITC,452.00,0.16,-1.71,12.70,10.78,30000.0,-4100.0,236218.0,-41.56,45.0,X-LC,2.15,5.0,-0.14,1.69,3.44,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,1.19,-37.63,113.07,32.89,54448.0,-29052.0,48154.0,-694.37,53.0,L-MC,6.35,259.0,-0.53,0.34,32.61,XR,NTT,BANKS
56,RAJESHEXPO,518.00,9.03,-60.20,151.46,0.07,82682.0,-82587.0,54590.0,1901.26,67.0,L-SC,2.62,268.0,-1.00,0.39,35.77,OX40N,NTT,JEWELLERY
6,ASIANTILES,137.00,4.94,-4.43,90.09,81.67,80770.0,-4155.0,89655.0,8072.22,71.0,L-SC,13.01,271.0,-0.05,0.64,73.62,XR,NTT,CERAMICS
50,MASFIN,398.61,1.50,-2.88,25.67,22.05,24428.0,-2820.0,95160.0,-16.44,52.0,H-SC,6.41,164.0,-0.12,0.68,38.32,XR,ATH,FINANCE
13,BSOFT,831.70,-0.27,-21.64,92.43,50.79,101868.0,-30442.0,110211.0,0.50,64.0,H-SC,6.87,151.0,-0.30,0.79,28.32,XR,ATH,IT


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BSOFT,831.7,-0.27,-21.64,92.43,50.79,101868.0,-30442.0,110211.0,0.50,64.0,H-SC,6.87,151.0,-0.30,0.79,28.32,XR,ATH,IT
56,RAJESHEXPO,518.0,9.03,-60.20,151.46,0.07,82682.0,-82587.0,54590.0,1901.26,67.0,L-SC,2.62,268.0,-1.00,0.39,35.77,OX40N,NTT,JEWELLERY
84,WIPRO,420.0,0.26,8.50,58.77,72.27,108047.0,14400.0,183848.0,-6.99,70.0,M-LC,3.06,99.0,0.13,1.32,15.20,XR,NTT,IT
6,ASIANTILES,137.0,4.94,-4.43,90.09,81.67,80770.0,-4155.0,89655.0,8072.22,71.0,L-SC,13.01,271.0,-0.05,0.64,73.62,XR,NTT,CERAMICS
41,INFY,1972.0,0.84,11.67,20.21,34.24,70615.0,36517.0,349405.0,-13.14,69.0,X-LC,4.89,2.0,0.52,2.50,20.92,X40,NTT,IT


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.37
1,25,44.87
2,50,76.54


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.91
MC    30.85
LC    24.21
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.73
X40      23.05
X40N     12.98
XR        9.83
XY25      9.07
AR        8.93
OX40N     7.61
SR        1.02
MH        0.75
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.35
H-SC    23.71
X-LC    20.76
M-SC    11.63
X-SC     8.05
H-MC     4.82
L-SC     1.52
M-MC     1.34
M-LC     1.32
H-LC     1.11
L-LC     1.02
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.27,-8.05,42.80
IT,13.12,-14.79,72.50
FINANCE,9.47,-20.46,72.85
MISC,7.26,-28.03,80.41
ELECTRICAL,5.99,-12.17,53.26
PAINTS,5.83,-13.57,30.43
INSURANCE,4.81,-0.79,35.28
PHARMA,4.11,-2.44,34.68
AUTO,2.84,-31.85,77.56


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3186079.0,21
AR,1324903.0,10
XR,1288872.0,13
X40,1003113.0,14
X40N,872403.0,9
OX40N,730116.0,10
XY25,374367.0,6
SR,278583.0,2
MH,78861.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3666561.0,25
M-SC,1434160.0,15
X-MC,1423756.0,16
X-LC,826730.0,11
X-SC,794658.0,8
H-MC,407060.0,3
L-SC,248701.0,3
M-LC,108047.0,1
H-LC,78077.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1281011.0      6
           AR         909200.0      5
M-SC       XY24       821056.0      6
H-SC       XR         784791.0      7
X-MC       X40        493592.0      7
           XY24       402730.0      3
           X40N       343011.0      4
H-SC       OX40N      334115.0      4
X-LC       X40        317720.0      5
M-SC       OX40N      313319.0      5
X-SC       X40N       307073.0      3
           XY24       295784.0      3
H-SC       SR         278583.0      2
X-LC       X40N       222319.0      2
H-MC       AR         213408.0      2
           XY24       193652.0      1
X-LC       XY24       191846.0      2
X-SC       X40        191801.0      2
X-MC       XY25       184423.0      2
M-SC       XR         175567.0      2
L-SC       XR         166019.0      2
M-SC       AR         124218.0      2
M-LC       XR         108047.0      1
X-LC       XY25        94845.0      2
L-SC       OX40N       82682.0      1
H-SC       MH          78861.0      1
H-LC       AR          78077.0      1
M-MC       XY25        58432.0      1
L-MC       XR          54448.0      1
L-LC       XY25        36667.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 26.0 seconds
